In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMolNotebooks
using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"
mol_type = "6r7m"

id_index = 1
separator = "."
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println("Error in $file")
        end
    end
end

In [3]:
println("Minimal configuration: ", argmin(min_Es))
for elem in evaluation_strings
    println(elem)
end

Minimal configuration: 23
1: 2.0 | 0.039053954 | 8180.531390115301, 8120.639093570557, 8120.639093570557 | 22.415324813974376
2: 2.0 | 0.016035756 | 8197.504819511309, 8109.453476956436, 8109.456202992612 | 28.67677949378188
3: 2.0 | 0.018456742 | 8202.668848612144, 8108.355383345988, 8127.034654182801 | 9.273109916541754
4: 2.0 | 0.012463227 | 8257.85548932352, 8100.641188938689, 8106.899218802393 | 1.1406207814281855
5: 2.0 | 0.0144451335 | 8171.214128028016, 8103.744145001545, 8109.223685408802 | 19.27582843646324
6: 2.0 | 0.0043075527 | 8161.945492317855, 8080.973876582009, 8085.4292716574255 | 0.938706069485428
7: 2.0 | 0.011415175 | 8173.958764852578, 8098.990059564388, 8107.534641382518 | 20.923163854502178
8: 2.0 | 0.0056008147 | 8223.867311456717, 8113.819989395824, 8133.972087693552 | 12.30926287793688
9: 2.0 | 0.009449994 | 8198.570213345718, 8077.045331194652, 8077.728487368631 | 0.989983051482274
10: 2.0 | 0.015068582 | 8324.49773200844, 8100.371684553767, 8102.39343151968

In [4]:
@load "$(folder)30.jld2" input output
display(MorphoMolNotebooks.interactive_simulation_analysis(merge!(input, Dict("mol_type" => "6r7m")), output))

GLMakie.Screen(...)